In [1]:
import cv2
import numpy as np
import pandas as pd
import requests

In [3]:
# 이미지 전처리 함수 호출

# 1. url주소 img로 변경
def get_normal_image(url):
    image_nparray = np.asarray(bytearray(requests.get(url).content), dtype=np.uint8)
    image = cv2.imdecode(image_nparray, cv2.IMREAD_COLOR)
    return image

# 2. 배경 제거 및 상품크기에 맞게 다시 자르기
def remove_background_and_crop(image):
    # 블러 처리 적용
    blurred_image = cv2.GaussianBlur(image, (15, 15), 0)

    # 초기 마스크 생성
    mask = np.zeros(image.shape[:2], np.uint8)

    # ROI(Region of Interest) 설정
    rect = (10, 10, image.shape[1]-10, image.shape[0]-10)

    # GrabCut 알고리즘 수행
    bgd_model = np.zeros((1, 65), np.float64)
    fgd_model = np.zeros((1, 65), np.float64)
    cv2.grabCut(blurred_image, mask, rect, bgd_model, fgd_model, 5, cv2.GC_INIT_WITH_RECT)

    # 마스크 수정
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

    # 이미지에 새로운 마스크 적용
    result = blurred_image * mask2[:, :, np.newaxis]

    # 좌표 구하기
    coords = np.column_stack(np.where(mask2 == 1))
    x_min, y_min = coords.min(axis=0)
    x_max, y_max = coords.max(axis=0)

    # 이미지 잘라내기
    result = result[x_min:x_max, y_min:y_max]
    
    return result


# 3. 정방형으로 맞추기
def resize_and_center(image):
    # 이미지의 가로, 세로 길이 가져오기
    height, width = image.shape[:2]

    # 더 긴 쪽의 길이를 구하기
    max_length = max(height, width)

    # 새로운 이미지 생성 (4차원)
    new_image = np.zeros((max_length, max_length, 3), dtype=np.uint8)

    # 중앙에 이미지 배치
    y_offset = (max_length - height) // 2
    x_offset = (max_length - width) // 2
    new_image[y_offset:y_offset+height, x_offset:x_offset+width] = image

    return new_image

# 4. 150*150 크기로 변환
def resize_image(image, target_size=(150, 150)):
    # 이미지 리사이즈
    resized_image = cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)
    return resized_image

In [4]:
img1 = get_normal_image('https://image.msscdn.net/images/goods_img/20230209/3070468/3070468_16759031955428_500.jpg')

cv2.imshow('img1', img1)
cv2.waitKey(0)

img1 = remove_background_and_crop(img1)
img1 = resize_and_center(img1)
final_img1 = resize_image(image = img1)
cv2.imshow('final_img1', final_img1)
cv2.waitKey(0)